In [9]:
import pandas as pd
import numpy as np
ven = pd.read_csv("C:/VS code projects/Road to Hack/auto_negotiator/Utilities/Vendor_yes.csv")

def displayMinVendor(ven):
    ven["Unfulfilled_len"] = ven['Unfulfilled'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)

    ven['Cost' + '_normalized'] = normalize_by_sum(ven['Cost'])
    ven['Days' + '_normalized'] = normalize_by_sum(ven['Days'])
    ven['Unfulfilled_len' + '_normalized'] = normalize_by_sum(ven['Unfulfilled_len'])

    
    ven['Overall'] = ven.apply(lambda row: 0.4 * row['Cost_normalized'] +
                                       0.35 * row['Days_normalized'] +
                                       0.25 * row['Unfulfilled_len_normalized'], axis=1)
    
    
    matching_row = ven[(ven['Cost'] == ven['Cost'].min()) & (ven['Days'] == ven['Days'].min())]

    if not matching_row.empty:
    # If a matching row exists, print the row
        print("Matching Row:")
        print(matching_row)

        return matching_row[['VendorID', 'Cost', 'Days', 'Unfulfilled']], (matching_row['Cost'].min(), matching_row['Days'].min())
    else:
    # If no matching row exists, print the minimum values
        min_values_Cost = ven['Cost'].min()
        min_values_Days = ven['Days'].min()
        print(f"Minimum Cost Offered: {min_values_Cost}, Mimimum Days Offered: {min_values_Days}")
    
        min_values = ven['Overall'].nsmallest(1)
        result_rows = ven[ven['Overall'].isin(min_values)]

        return result_rows[['VendorID', 'Cost', 'Days', 'Unfulfilled']], (min_values_Cost, min_values_Days)
    
def normalize_by_sum(column):
    normalized_column = column / column.sum()
    return normalized_column

In [39]:
displayMinVendor(ven)

Minimum Cost Offered: 3, Mimimum Days Offered: 14


(   VendorID  Cost  Days Unfulfilled
 2      4819     3    20         NaN,
 (3, 14))

In [57]:
offer = {

"requirements_satisfied": True,

"unfullfiled_requirements": None,

"quotation": 3,

"procurement_days": 18,

}

In [24]:
def get_counter_offer(offer):
    ven = pd.read_csv("C:/VS code projects/Road to Hack/auto_negotiator/Utilities/Vendor_yes.csv")
    ## Check if the requirements are satisfied
    if offer["requirements_satisfied"]:
        ## Convert offer to dataframe
        dataframe = pd.DataFrame(np.array([['NEW'] + list(offer.values())])[:,[0,-2,-1,1,2]], columns=['VendorID', 'Cost', 'Days', 'CanFulfill', 'Unfulfilled'],index=[len(ven)])
        #Replace True with T and False with F
        dataframe['CanFulfill'] = dataframe['CanFulfill'].astype(str).str[0].str.upper()
        
        ## Append the offer to the vendor dataframe
        ven = ven.append(dataframe, ignore_index=True)
        ## Convert cost and days to numeric
        ven['Cost'] = pd.to_numeric(ven['Cost'])
        ven['Days'] = pd.to_numeric(ven['Days'])

        ## Get minimum vendor
        min_vendor, min_values = displayMinVendor(ven)

        if min_vendor['VendorID'].iloc[0] == 'NEW':
            ## Check if min_vendor is better than min_values
            if min_vendor['Cost'].iloc[0] > min_values[0] or min_vendor['Days'].iloc[0] > min_values[1]:
                ## Create a new offer in {'new_cost':3,'new_days':14} format
                return True, {'new_cost':min(min_vendor['Cost'].iloc[0], min_values[0]),'new_days':min(min_vendor['Days'].iloc[0], min_values[1])}, {'previous_cost':min_vendor['Cost'].iloc[0],'previous_days':min_vendor['Days'].iloc[0]}
            else:
                return False, {}
        else:
            False, {}
        

In [28]:
import json
import pandas as pd
import numpy as np
offer = json.loads(json.load(open('C:/VS code projects/Road to Hack/auto_negotiator/Utilities/procurement.json','r')))
#Checkng conditions
if offer['requirements_satisfied'] == 'true':
    offer['requirements_satisfied'] = True
if offer['requirements_satisfied'] == 'false':
    offer['requirements_satisfied'] = False
if offer['unfullfiled_requirements'] == 'NA':
    offer['unfullfiled_requirements'] = None
if offer['quotation'] == 'NA':
    offer['quotation'] = None
if offer['procurement_days'] == 'NA':
    offer['procurement_days'] = None

In [29]:
offer

{'requirements_satisfied': True,
 'unfullfiled_requirements': None,
 'quotation': '3',
 'procurement_days': '18'}

In [30]:
get_counter_offer(offer)

Minimum Cost Offered: 3, Mimimum Days Offered: 14


C:\Users\aniru\AppData\Local\Temp\ipykernel_10616\241073260.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ven = ven.append(dataframe, ignore_index=True)


(True,
 {'new_cost': 3, 'new_days': 14},
 {'previous_cost': 3, 'previous_days': 18})